In [1]:
import torch

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from transformers import TrainingArguments, Trainer

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import evaluate

2024-02-02 20:32:19.176314: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## GPT Model

### Pipeline Usage

In [ ]:
gpt = pipeline('text-generation', model='openai-gpt')

In [ ]:
prompt = "Once upon a time, there was a mad king"
answer = gpt(prompt, max_length=30)
print(answer[0]["generated_text"])

### Importing the model as Classifier and the tokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("openai-gpt")
model = AutoModelForSequenceClassification.from_pretrained("openai-gpt")

Some weights of OpenAIGPTForSequenceClassification were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Adding a pad token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Set the pad_token_id in the model configuration
model.config.pad_token_id = tokenizer.pad_token_id

# Make sure the model is aware of the new tokens
model.resize_token_embeddings(len(tokenizer))

Embedding(40479, 768)

In [4]:
inputs = tokenizer("Hello, I'm testing the first gpt", return_tensors="pt")
inputs

{'input_ids': tensor([[3570,  240,  249,  256,  258, 8448,  481,  929,   30, 1992]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [5]:
inputs = tokenizer("Hello, I'm testing the first gpt", return_tensors="pt")
outputs = model(**inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0175, -0.0210]], grad_fn=<IndexBackward0>), hidden_states=None, attentions=None)

In [6]:
prediction = torch.argmax(outputs.logits)
prediction

tensor(0)

## SST-2 Dataset

In [7]:
dataset = load_dataset("sst2")
dataset

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

In [8]:
print(dataset["train"]["sentence"][0])
print(dataset["train"]["label"][0])

hide new secretions from the parental units 
0


In [9]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [10]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

In [11]:
small_train_dataset

Dataset({
    features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask'],
    num_rows: 100
})

## Finetuning

In [12]:
training_args = TrainingArguments(output_dir="test_trainer",
                                  num_train_epochs=2,
                                  learning_rate=1e-3,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch")

In [13]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

In [15]:
trainer.train()

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.0, 'eval_accuracy': 0.0, 'eval_runtime': 6.7173, 'eval_samples_per_second': 14.887, 'eval_steps_per_second': 1.935, 'epoch': 1.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.0, 'eval_accuracy': 0.0, 'eval_runtime': 6.6434, 'eval_samples_per_second': 15.052, 'eval_steps_per_second': 1.957, 'epoch': 2.0}
{'train_runtime': 126.3124, 'train_samples_per_second': 1.583, 'train_steps_per_second': 0.206, 'train_loss': 1.1670847672682543, 'epoch': 2.0}


TrainOutput(global_step=26, training_loss=1.1670847672682543, metrics={'train_runtime': 126.3124, 'train_samples_per_second': 1.583, 'train_steps_per_second': 0.206, 'train_loss': 1.1670847672682543, 'epoch': 2.0})